In [1]:
import pandas as pd
from scipy import stats

## Импорт данных для анализа

In [2]:
contracts = pd.read_excel('CASE_CONTRACTS.xlsx')
losses = pd.read_excel('CASE_LOSSES.xlsx')
experiment_groups = pd.read_excel('experiment_group.xlsx')

In [3]:
contracts.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000


In [4]:
experiment_groups.head()

,client_id,group,experiment_id
0,10001718,control,policy_pricing
1,1017180395,control,policy_pricing
2,1017178047,control,policy_pricing
3,1017174614,control,policy_pricing
4,1017174071,control,policy_pricing


In [5]:
losses.head()

,loss_id,client_id,loss_name,loss_payout_amt
0,1,10000713193,Оказание медицинской помощи,100000
1,2,100014656,Оказание медицинской помощи,50000
2,3,10006545463,Оказание медицинской помощи,30000
3,4,10006611194,Оказание медицинской помощи,70000
4,5,1001091597,Оказание медицинской помощи,60000


### Объединение данных

In [6]:
data = pd.merge(experiment_groups, contracts, on='client_id', how='left')
data = pd.merge(data, losses, on='client_id', how='left')

In [7]:
data.head()

,client_id,group,experiment_id,contract_id,contract_num,product_name,contract_status,currency_name,duration,country,price,insurance_amount,loss_id,loss_name,loss_payout_amt
0,10001718,control,policy_pricing,2.123569e+08,ТТЕ7227715*****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
1,1017180395,control,policy_pricing,2.135234e+07,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
2,1017178047,control,policy_pricing,2.147454e+09,ТМБЕ579 № *****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
3,1017174614,control,policy_pricing,2.135192e+07,ТМГ1535Е7 *****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
4,1017174071,control,policy_pricing,2.162434e+08,ТТЕ7227777*****,Страхование путешественников,Завершен,Российский рубль,10.0,Канада,1096.0,2000000.0,NaN,NaN,NaN


### Добавление столбца 'price_rub' с курсом рубля 80

In [8]:
data['price_rub'] = data['price'] * 80

### Разделение на тестовую и контрольную группы

In [9]:
group_test = data[data['group'] == 'test']
group_control = data[data['group'] == 'control']

In [10]:
# 1. Средняя цена полиса
mean_price_test = group_test['price_rub'].mean()
mean_price_control = group_control['price_rub'].mean()

In [11]:
# 2. Конверсия в оформление
conversion_test = len(group_test[~group_test['contract_id'].isna()]) / len(group_test)
conversion_control = len(group_control[~group_control['contract_id'].isna()]) / len(group_control)

In [12]:
# 3. Убыточность
total_losses_test = group_test['loss_payout_amt'].sum()
total_premiums_test = group_test['price_rub'].sum()
loss_ratio_test = total_losses_test / total_premiums_test if total_premiums_test else 0

total_losses_control = group_control['loss_payout_amt'].sum()
total_premiums_control = group_control['price_rub'].sum()
loss_ratio_control = total_losses_control / total_premiums_control if total_premiums_control else 0

## Вывод и анализ результатов

In [13]:
print("Группа A (традиционный подход)")
print(f"Средняя цена полиса: {mean_price_test:.2f} RUB")
print(f"Конверсия: {conversion_test:.2%}")
print(f"Убыточность: {loss_ratio_test:.2%}")

print("\nГруппа B (кластеризация)")
print(f"Средняя цена полиса: {mean_price_control:.2f} RUB")
print(f"Конверсия: {conversion_control:.2%}")
print(f"Убыточность: {loss_ratio_control:.2%}")


Группа A (традиционный подход)
Средняя цена полиса: 370896.27 RUB
Конверсия: 26.44%
Убыточность: 0.56%

Группа B (кластеризация)
Средняя цена полиса: 289865.78 RUB
Конверсия: 32.25%
Убыточность: 1.13%


##### Убыточность группы B (1.13%) значительно выше, чем у группы A (0.56%). Это ключевой фактор, влияющий на прибыльность бизнеса.

Хотя группа B (кластеризация) демонстрирует более высокую конверсию, ее убыточность значительно выше, чем у группы A.  Учитывая, что p-value для сравнения средней цены полиса меньше 0.05, мы можем сделать вывод о статистически значимой разнице в средней цене между группами. 

Использование традиционного подхода может привести к потере некоторой части клиентов, которые могли бы быть привлечены более низкой ценой в группе B. Однако, учитывая значительно более высокую убыточность группы B,  этот риск оправдан.

#### Статистический тест для сравнения средних значений цены полиса

In [14]:
t_statistic, p_value = stats.ttest_ind(group_test['price_rub'], group_control['price_rub'], nan_policy='omit')
print(f"\nP-value для сравнения средней цены полиса: {p_value:.3f}")


P-value для сравнения средней цены полиса: 0.002


Низкое значение p-value (0.002) подтверждает статистически значимую разницу в средней цене полиса между группами.

# --- Бизнес-рекомендация ---

In [15]:
print("""
На основе предоставленных данных, рекомендуется продолжить использовать традиционный подход 
к ценообразованию (группа A)
""")


На основе предоставленных данных, рекомендуется продолжить использовать традиционный подход 
к ценообразованию (группа A)



# --- Бизнес-рекомендация ---